## **Setting up Development Environment**

## **Importing Libraries**

In [2]:
import pandas as pd

import numpy as np
import re


from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from transformers import AutoTokenizer, AutoModel


import pickle
import os

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
import os
print(os.getcwd())  # Prints the current working directory
os.chdir("C:\\Users\\vijai\\NLP_Project")
print(os.getcwd())

c:\Users\vijai\NLP_Project
C:\Users\vijai\NLP_Project


## **Loading Data**

In [8]:
dataframe = pd.read_csv("hatewords_tamil_train.csv")

In [9]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,விஜய் சேதுபதி,NOT


In [10]:
dataframe.describe()

,id,text,category
count,200,200,200
unique,200,200,2
top,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
freq,1,1,161


In [11]:
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataframe.shape

(200, 3)

In [13]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [14]:
dataframe.shape

(200, 3)

In [15]:
text = dataframe['text']

In [16]:
text

0      திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1      இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2      மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3      மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                                          விஜய் சேதுபதி
                             ...                        
195    ஜே.குருவின் தம்பிகள் சார்பாக படம் வெற்றி பெற வ...
196    அகரம் பவுண்டேஷன் மூலம் நிறைய பேர் கல்விக்கு உத...
197    இப்படம் எப்போது வந்தாலும் எங்க ஊரை சேர்ந்த 25-...
198    அய்யா சத்திரிய குஞ்சுகலா உங்ககிட்ட  உள்ள சொத்த...
199    மிகவும் அருமையான படம் தயவுசெய்து அனைவரும் தியே...
Name: text, Length: 200, dtype: object

In [17]:
label = dataframe['category']

## **Data Preprocessing**

In [18]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0])

In [19]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       'மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது......  என்னா சமூகம் டா இது  தூ...........',
       'விஜய் சேதுபதி', 'உணமையாவே இது சைக்கோ படம் தான் ஒன்னுமே புரில',
       'முத்தையா அண்ணனுக்கும் சாதி சாயம் பூசினார்கள் இப்பொழுது மோகன் அண்ணனுக்கும் சாதி சாயம் பூசி விடுவார்கள்',
       'தமிழ்நாட்டில் மாற்று மொழி பேசும் அனைத்து மக்கள் சார்பில் படம் வெற்றி பெற வாத்துக்கள்..',
       'எங்களுக்கு மண்ணு பொண்ணு இரண்டுமே முக்கியம் அதில் யாரு கையை வச்சாலும் கையை வெட்டுவோம்',
       'படம் அழகாக இருக்குங்க  அத விட எதார்த்தமாயிருக்குங்க',
       ', இந்த படம் மாபெரும் வெற்றி பெற்று தமிழ் சினிமாவில் ஆதிக்கம் செலுத்தும் ,,,இது போன்ற மேலும் பல படங்களை இயக்க வாழ்த்துக்கள்',
       'நாங்க இருக்கோம் மோகன்  அண்ணா ,கவலை படாதீங்க',
       'தமிழக மக்கள் சார்பாக வாழ்த்துக்

### Setting Stopwords


In [20]:
with open('tamil_stopwords.txt', encoding = 'utf-8') as f:
    tamil_stopwords = f.readlines()
    for i in range(len(tamil_stopwords)):
        tamil_stopwords[i] = re.sub('\n','',tamil_stopwords[i])
stopwords = tamil_stopwords

In [21]:
# Function for removing stop words
def stopwords_remove(text):
    # Appending words which are not stop words
    text = text.split(" ")
    removed = [s for s in text if s not in stopwords]  
    return removed

In [22]:
text = [stopwords_remove(s) for s in text]
for i in range(len(text)):
  text[i] = (" ".join(text[i]))

## **Feature Extraction**

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", use_fast=False)
model = AutoModel.from_pretrained("ai4bharat/indic-bert")


In [ ]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [21]:
print(tokenized_input)

{'input_ids': tensor([[     2,   4436, 139013, 105466,   1385,  30231,  40291,  39555,  82156,
         150078, 194115,   5343, 104252,  82076,   3587, 123516,  18248,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [22]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0103, -0.0047, -0.0161,  ..., -0.0298, -0.0208, -0.0208],
         [ 0.1137,  0.0977, -0.4158,  ..., -0.1404,  0.3146,  0.1815],
         [ 0.5111, -0.4328, -0.0232,  ...,  0.3004,  0.1211, -0.0870],
         ...,
         [-0.3719, -0.1325, -0.2703,  ...,  0.4747, -0.1124,  0.3068],
         [-0.1099,  0.0255, -0.1874,  ..., -0.2515, -0.3389, -0.4451],
         [-0.0103, -0.0047, -0.0161,  ..., -0.0298, -0.0208, -0.0208]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-5.2173e-02,  5.5841e-02,  2.0672e-02,  9.0904e-04,  5.0800e-02,
          8.3491e-02,  4.0969e-02, -4.8967e-03, -1.1270e-02,  6.2111e-02,
         -1.0971e-02,  4.8428e-02,  1.4276e-02, -1.8487e-02,  4.9799e-02,
         -1.4342e-02, -4.2831e-02, -5.2692e-04, -1.1993e-01, -8.3791e-02,
          7.8855e-02,  7.2183e-02, -3.5856e-02, -9.7793e-02,  2.6021e-03,
         -3.4501e-02, -5.8019e-02, -4.5339e-03,  2.4570e-02,  5.5002e-03,
          2

In [23]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [24]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [25]:
# Directory path
output_dir = "./IndicBert-Stopword-Embedded-TrainData/"
os.makedirs(output_dir, exist_ok=True)

# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./IndicBert-Stopword-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./IndicBert-Stopword-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./IndicBert-Stopword-Embedded-TrainData/output1.pickle done
./IndicBert-Stopword-Embedded-TrainData/output2.pickle done


In [26]:
# Loading Embedded Input Data from Disk

output = []

for i in range(2):
    file_name = "./IndicBert-Stopword-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./IndicBert-Stopword-Embedded-TrainData/output1.pickle done
./IndicBert-Stopword-Embedded-TrainData/output2.pickle done


In [27]:
X = np.array(X)
X.shape

(200, 1, 768)

In [28]:
X = X.reshape(-1, 768)
X.shape

(200, 768)

In [29]:
y = label
y.shape

(200,)

## **Train Test Split**

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(160, 768)

In [32]:
X_val.shape

(40, 768)

## **Logistic Regression**

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred,zero_division=1))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        33
           1       1.00      0.00      0.00         7

    accuracy                           0.82        40
   macro avg       0.91      0.50      0.45        40
weighted avg       0.86      0.82      0.75        40



## **Naive Bayes**

In [34]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred,zero_division=1))

              precision    recall  f1-score   support

           0       0.86      0.55      0.67        33
           1       0.21      0.57      0.31         7

    accuracy                           0.55        40
   macro avg       0.53      0.56      0.49        40
weighted avg       0.74      0.55      0.60        40



## KNN

In [46]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.82      0.97      0.89        33
           1       0.00      0.00      0.00         7

    accuracy                           0.80        40
   macro avg       0.41      0.48      0.44        40
weighted avg       0.68      0.80      0.73        40



## SVM

In [45]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred,zero_division=1))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        33
           1       1.00      0.00      0.00         7

    accuracy                           0.82        40
   macro avg       0.91      0.50      0.45        40
weighted avg       0.86      0.82      0.75        40



## Random Forest

In [73]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred,zero_division=1))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90        33
           1       0.50      0.14      0.22         7

    accuracy                           0.82        40
   macro avg       0.67      0.56      0.56        40
weighted avg       0.78      0.82      0.78        40



## **Loading Testing Data**

In [47]:
dataframe_test = pd.read_csv(
    "hatewords_tamil_test_nolabels.csv"
)
dataframe_test.head()

,id,text
0,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....
1,tam_test_600,தமிழகத்தில் நாடக காதல் கும்பலை அடியோடு கருவறுக...
2,tam_test_601,பாளையக்கார சமுகம் சார்பாக இத்திரைபடம் வெற்றி ப...
3,tam_test_602,தேவேந்திர குல வேளாளர் சமுதாயம் சரர்ப்பாக வாழ்த...
4,tam_test_603,தேவராட்டம் பார்த்திருப்பீங்க இது வன்னியர் ...


In [48]:
dataframe_test.describe()

,id,text
count,52,52
unique,52,52
top,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....
freq,1,1


In [49]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [50]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

In [51]:
text_test = [stopwords_remove(s) for s in text_test]
for i in range(len(text_test)):
  text_test[i] = (" ".join(text_test[i]))

## **Test Feature Extraction**

In [52]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

C:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [53]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [54]:
print(tokenized_input)

{'input_ids': tensor([[     2,  14359, 199677,   1385,   1385,   5010,   5448,  22227,   1471,
           1471,   3534,   4387,   7859,   7859,   1299,  21839,   5013, 141481,
           1385,   4146, 188289,   1471,   1173,  17275,   5798,  20471,  56923,
          45689,  95464,  21585,   5448,   1385,   1385,   4146,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [55]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0015, -0.0114, -0.0036,  ..., -0.0254, -0.0093, -0.0246],
         [ 0.3723, -0.2365, -0.0742,  ..., -0.1618,  0.1132, -0.0238],
         [ 0.7442, -0.3239, -0.1377,  ...,  0.0884,  0.4298, -0.3640],
         ...,
         [-0.1053,  0.4545, -0.2647,  ..., -0.1936, -0.0931, -0.6346],
         [ 0.3120,  0.2352, -0.2636,  ..., -0.0855,  0.0102, -0.3817],
         [-0.0015, -0.0114, -0.0036,  ..., -0.0254, -0.0093, -0.0246]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-5.2554e-02,  5.7768e-02,  1.5031e-02, -1.6805e-03,  4.9569e-02,
          8.9458e-02,  4.4586e-02, -8.2878e-03, -1.7834e-02,  6.2598e-02,
         -1.2995e-02,  4.9803e-02,  2.0443e-02, -2.1443e-02,  4.6841e-02,
         -1.6773e-02, -4.4487e-02, -1.0702e-03, -1.2682e-01, -8.3631e-02,
          8.2461e-02,  7.2334e-02, -3.3373e-02, -9.8802e-02, -5.4013e-04,
         -3.3796e-02, -6.0080e-02, -5.2718e-03,  1.9947e-02,  6.5769e-03,
          2

In [56]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [57]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [58]:
# Directory path
output_dir = "./IndicBert-Stopword-Embedded-TestData/"
os.makedirs(output_dir, exist_ok=True)

# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./IndicBert-Stopword-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./IndicBert-Stopword-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./IndicBert-Stopword-Embedded-TestData/output1.pickle done


In [59]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(1):
    file_name = "./IndicBert-Stopword-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./IndicBert-Stopword-Embedded-TestData/output1.pickle done


In [60]:
X_test = np.array(X_test)
X_test.shape

(52, 1, 768)

In [61]:
X_test = X_test.reshape(-1, 768)
X_test.shape

(52, 768)

## **Getting Predictions**

In [77]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

#KNN
knn_test_pred=knn.predict(X_test)

#Random Forest
rf_test_pred=rf.predict(X_test)

#Support Vector Machine
svm_test_pred=svm.predict(X_test)

## **Evaluating the Predictions**

In [63]:
dataframe_eval = pd.read_csv(
    "hatewords_tamil_test_labels.csv"
)

In [64]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [67]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.88      0.72      0.79        40
           1       0.42      0.67      0.52        12

    accuracy                           0.71        52
   macro avg       0.65      0.70      0.66        52
weighted avg       0.77      0.71      0.73        52



In [79]:
print('SVM')
print(classification_report(y_test, svm_test_pred,zero_division=1))

SVM
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        40
           1       1.00      0.00      0.00        12

    accuracy                           0.77        52
   macro avg       0.88      0.50      0.43        52
weighted avg       0.82      0.77      0.67        52



In [66]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred,zero_division=1))

Logistic Regression
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        40
           1       1.00      0.00      0.00        12

    accuracy                           0.77        52
   macro avg       0.88      0.50      0.43        52
weighted avg       0.82      0.77      0.67        52



In [68]:
print('KNN')
print(classification_report(y_test, knn_test_pred))

KNN
              precision    recall  f1-score   support

           0       0.78      1.00      0.88        40
           1       1.00      0.08      0.15        12

    accuracy                           0.79        52
   macro avg       0.89      0.54      0.52        52
weighted avg       0.83      0.79      0.71        52



In [75]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred,zero_division=1))

Random Forest
              precision    recall  f1-score   support

           0       0.81      0.97      0.89        40
           1       0.75      0.25      0.38        12

    accuracy                           0.81        52
   macro avg       0.78      0.61      0.63        52
weighted avg       0.80      0.81      0.77        52



## **Saving the Predictions**

In [80]:
# Naive Bayes

predictions = le.inverse_transform(nb_test_pred)

dataframe_test_nb = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_nb = pd.DataFrame(dataframe_test_nb)
dataframe_test_nb.to_csv("Output/IndicBert-Stopword-NaiveBayes.csv")
dataframe_test_nb

,id,text,category
0,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....,OFF
1,tam_test_600,தமிழகத்தில் நாடக காதல் கும்பலை அடியோடு கருவறுக...,NOT
2,tam_test_601,பாளையக்கார சமுகம் சார்பாக இத்திரைபடம் வெற்றி ப...,NOT
3,tam_test_602,தேவேந்திர குல வேளாளர் சமுதாயம் சரர்ப்பாக வாழ்த...,NOT
4,tam_test_603,தேவராட்டம் பார்த்திருப்பீங்க இது வன்னியர் ...,NOT
5,tam_test_604,படம் வெற்றி பெற பறைய்யர் குலம் சாம்பவர்ரகுலம் ...,OFF
6,tam_test_605,ரஞ்சித் மற்றும் குருமா க்கு சரியான செருப்படி.. .,NOT
7,tam_test_606,அட ஜாதி வெறி பிடிச்சவங்கலா எதன பேர் டா ஜாதி சர...,OFF
8,tam_test_607,படம் நல்லா இருந்தா பாக்கலாம் போலயே !,NOT
9,tam_test_608,மனித குலத்தின் சார்பாக படம் மாபெரும் வெற்றி பெ...,NOT


In [84]:
# SVM

predictions = le.inverse_transform(svm_test_pred)

dataframe_test_svm = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_svm = pd.DataFrame(dataframe_test_svm)
dataframe_test_svm.to_csv("Output/IndicBert-Stopword-SupportVectorMachine.csv")
dataframe_test_svm

,id,text,category
0,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....,NOT
1,tam_test_600,தமிழகத்தில் நாடக காதல் கும்பலை அடியோடு கருவறுக...,NOT
2,tam_test_601,பாளையக்கார சமுகம் சார்பாக இத்திரைபடம் வெற்றி ப...,NOT
3,tam_test_602,தேவேந்திர குல வேளாளர் சமுதாயம் சரர்ப்பாக வாழ்த...,NOT
4,tam_test_603,தேவராட்டம் பார்த்திருப்பீங்க இது வன்னியர் ...,NOT
5,tam_test_604,படம் வெற்றி பெற பறைய்யர் குலம் சாம்பவர்ரகுலம் ...,NOT
6,tam_test_605,ரஞ்சித் மற்றும் குருமா க்கு சரியான செருப்படி.. .,NOT
7,tam_test_606,அட ஜாதி வெறி பிடிச்சவங்கலா எதன பேர் டா ஜாதி சர...,NOT
8,tam_test_607,படம் நல்லா இருந்தா பாக்கலாம் போலயே !,NOT
9,tam_test_608,மனித குலத்தின் சார்பாக படம் மாபெரும் வெற்றி பெ...,NOT


In [81]:
# Logistic Regression

predictions = le.inverse_transform(lr_test_pred)

dataframe_test_lr = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_lr = pd.DataFrame(dataframe_test_lr)
dataframe_test_lr.to_csv("Output/IndicBert-Stopword-LogisticRegression.csv")
dataframe_test_lr

,id,text,category
0,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....,NOT
1,tam_test_600,தமிழகத்தில் நாடக காதல் கும்பலை அடியோடு கருவறுக...,NOT
2,tam_test_601,பாளையக்கார சமுகம் சார்பாக இத்திரைபடம் வெற்றி ப...,NOT
3,tam_test_602,தேவேந்திர குல வேளாளர் சமுதாயம் சரர்ப்பாக வாழ்த...,NOT
4,tam_test_603,தேவராட்டம் பார்த்திருப்பீங்க இது வன்னியர் ...,NOT
5,tam_test_604,படம் வெற்றி பெற பறைய்யர் குலம் சாம்பவர்ரகுலம் ...,NOT
6,tam_test_605,ரஞ்சித் மற்றும் குருமா க்கு சரியான செருப்படி.. .,NOT
7,tam_test_606,அட ஜாதி வெறி பிடிச்சவங்கலா எதன பேர் டா ஜாதி சர...,NOT
8,tam_test_607,படம் நல்லா இருந்தா பாக்கலாம் போலயே !,NOT
9,tam_test_608,மனித குலத்தின் சார்பாக படம் மாபெரும் வெற்றி பெ...,NOT


In [82]:
# KNN

predictions = le.inverse_transform(knn_test_pred)

dataframe_test_knn = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_knn = pd.DataFrame(dataframe_test_knn)
dataframe_test_knn.to_csv("Output/IndicBert-Stopword-KNN.csv")
dataframe_test_knn

,id,text,category
0,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....,NOT
1,tam_test_600,தமிழகத்தில் நாடக காதல் கும்பலை அடியோடு கருவறுக...,NOT
2,tam_test_601,பாளையக்கார சமுகம் சார்பாக இத்திரைபடம் வெற்றி ப...,NOT
3,tam_test_602,தேவேந்திர குல வேளாளர் சமுதாயம் சரர்ப்பாக வாழ்த...,NOT
4,tam_test_603,தேவராட்டம் பார்த்திருப்பீங்க இது வன்னியர் ...,NOT
5,tam_test_604,படம் வெற்றி பெற பறைய்யர் குலம் சாம்பவர்ரகுலம் ...,NOT
6,tam_test_605,ரஞ்சித் மற்றும் குருமா க்கு சரியான செருப்படி.. .,NOT
7,tam_test_606,அட ஜாதி வெறி பிடிச்சவங்கலா எதன பேர் டா ஜாதி சர...,OFF
8,tam_test_607,படம் நல்லா இருந்தா பாக்கலாம் போலயே !,NOT
9,tam_test_608,மனித குலத்தின் சார்பாக படம் மாபெரும் வெற்றி பெ...,NOT


In [83]:
# Random Forest

predictions = le.inverse_transform(rf_test_pred)

dataframe_test_rf = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_rf = pd.DataFrame(dataframe_test_rf)
dataframe_test_rf.to_csv("Output/IndicBert-Stopword-RandomForest.csv")
dataframe_test_rf

,id,text,category
0,tam_test_599,டிஸ்லைக்கு போட்டவனெல்லாம் தண்ணிய குடிச்சிக்கோ....,OFF
1,tam_test_600,தமிழகத்தில் நாடக காதல் கும்பலை அடியோடு கருவறுக...,NOT
2,tam_test_601,பாளையக்கார சமுகம் சார்பாக இத்திரைபடம் வெற்றி ப...,NOT
3,tam_test_602,தேவேந்திர குல வேளாளர் சமுதாயம் சரர்ப்பாக வாழ்த...,NOT
4,tam_test_603,தேவராட்டம் பார்த்திருப்பீங்க இது வன்னியர் ...,NOT
5,tam_test_604,படம் வெற்றி பெற பறைய்யர் குலம் சாம்பவர்ரகுலம் ...,NOT
6,tam_test_605,ரஞ்சித் மற்றும் குருமா க்கு சரியான செருப்படி.. .,NOT
7,tam_test_606,அட ஜாதி வெறி பிடிச்சவங்கலா எதன பேர் டா ஜாதி சர...,OFF
8,tam_test_607,படம் நல்லா இருந்தா பாக்கலாம் போலயே !,NOT
9,tam_test_608,மனித குலத்தின் சார்பாக படம் மாபெரும் வெற்றி பெ...,NOT
